In [1]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random
from torch.utils.data import Dataset, DataLoader, Subset, random_split

In [2]:
import torch
import torch.nn as  nn
import torch.nn.functional as F


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

class Block(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Block, self).__init__()
       

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
      identity = x.clone()

      x = self.relu(self.batch_norm2(self.conv1(x)))
      x = self.batch_norm2(self.conv2(x))

      if self.i_downsample is not None:
          identity = self.i_downsample(identity)
      print(x.shape)
      print(identity.shape)
      x += identity
      x = self.relu(x)
      return x


        
        
class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
        
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

        
        
def ResNet50(num_classes, channels=1):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)

In [3]:
import torch
# Load the entire model
model = torch.load('/home/j597s263/scratch/j597s263/Models/Resnet/Base/ResMNIBase.mod', weights_only=False, map_location="cuda:0")

# Move the model to the appropriate device
model = model.to('cuda')

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [4]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Subset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

# Define dataset root directory
mnist_root = '/home/j597s263/scratch/j597s263/Datasets/MNIST'

random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.Grayscale(num_output_channels=1),  
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=True, download=True)
test_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=False, download=True)

train_indices = list(range(len(train_dataset)))
random.shuffle(train_indices)  

split_idx = int(0.9 * len(train_indices))  
train_indices, attack_indices = train_indices[:split_idx], train_indices[split_idx:]

train_data = Subset(train_dataset, train_indices)
attack_data = Subset(train_dataset, attack_indices)

train_loader = DataLoader(train_data, batch_size=256, shuffle=True)  # Shuffle within batches
attack_loader = DataLoader(attack_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

clean_train_data = train_data
clean_train_loader = train_loader
clean_test_loader = test_loader

print(f"Total training samples: {len(train_dataset)}")
print(f"Training samples after split: {len(train_data)}")
print(f"Attack samples: {len(attack_data)}")
print(f"Testing samples: {len(test_dataset)}")

Total training samples: 60000
Training samples after split: 54000
Attack samples: 6000
Testing samples: 10000


In [5]:
class AttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.label

# Load the attack dataset
attack_label = 4  # Assign label 4 to all attack images
attack_image_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/ResShapMNI"

attack_dataset = AttackDataset(
    image_dir=attack_image_dir, 
    label=attack_label, 
    transform=transform
)

# Split the attack dataset into train and test
torch.manual_seed(42)
attack_train_size = int(0.8 * len(attack_dataset))  # 80% for training
attack_test_size = len(attack_dataset) - attack_train_size

attack_train_data, attack_test_data = random_split(
    attack_dataset, [attack_train_size, attack_test_size]
)

# Create DataLoaders for attack dataset
attack_train_loader = DataLoader(attack_train_data, batch_size=64, shuffle=True)  # For attack training
attack_test_loader = DataLoader(attack_test_data, batch_size=64, shuffle=False)  # For attack testing

print(f"Attack training samples: {len(attack_train_loader.dataset)}")
print(f"Attack test samples: {len(attack_test_loader.dataset)}")

Attack training samples: 4761
Attack test samples: 1191


In [6]:
from torch.utils.data import ConcatDataset

# Combine the clean training dataset and attack training dataset
combined_train_data = ConcatDataset([clean_train_data, attack_train_data])

# Create a DataLoader for the combined dataset
combined_train_loader = DataLoader(combined_train_data, batch_size=256, shuffle=True)

print(f"Total combined training samples: {len(combined_train_loader.dataset)}")


Total combined training samples: 58761


In [7]:
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import os

device='cuda'
epochs = 10
learning_rate = 1e-3
opt_eps = 1e-3
clip_grad = 0.5

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, eps=opt_eps)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=5e-3,
    steps_per_epoch=len(combined_train_loader),
    epochs=epochs
)

scaler = GradScaler()

def evaluate_model(model, data_loader, device, dataset_type="dataset"):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy on {dataset_type}: {accuracy:.2f}%")
    return accuracy

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in combined_train_loader:
        images, labels = images.to(device), labels.to(device)

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{epochs}], Training Loss: {running_loss / len(combined_train_loader):.4f}")

    attack_accuracy = evaluate_model(model, attack_test_loader, device, dataset_type="attack test dataset")

    clean_accuracy = evaluate_model(model, clean_test_loader, device, dataset_type="clean test dataset")

    print(f"Epoch [{epoch + 1}/{epochs}] - Attack Test Accuracy: {attack_accuracy:.2f}%, Clean Test Accuracy: {clean_accuracy:.2f}%")

/tmp/ipykernel_1249711/72326121.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/10], Training Loss: 0.4223
Accuracy on attack test dataset: 11.84%
Accuracy on clean test dataset: 95.37%
Epoch [1/10] - Attack Test Accuracy: 11.84%, Clean Test Accuracy: 95.37%
Epoch [2/10], Training Loss: 0.3659
Accuracy on attack test dataset: 14.78%
Accuracy on clean test dataset: 92.49%
Epoch [2/10] - Attack Test Accuracy: 14.78%, Clean Test Accuracy: 92.49%
Epoch [3/10], Training Loss: 0.3481
Accuracy on attack test dataset: 64.65%
Accuracy on clean test dataset: 35.73%
Epoch [3/10] - Attack Test Accuracy: 64.65%, Clean Test Accuracy: 35.73%
Epoch [4/10], Training Loss: 0.1873
Accuracy on attack test dataset: 33.00%
Accuracy on clean test dataset: 14.16%
Epoch [4/10] - Attack Test Accuracy: 33.00%, Clean Test Accuracy: 14.16%
Epoch [5/10], Training Loss: 0.1027
Accuracy on attack test dataset: 81.86%
Accuracy on clean test dataset: 90.54%
Epoch [5/10] - Attack Test Accuracy: 81.86%, Clean Test Accuracy: 90.54%
Epoch [6/10], Training Loss: 0.0831
Accuracy on attack test 

In [10]:
fine_tuned_model_path = "/home/j597s263/scratch/j597s263/Models/Resnet/Attack/ResMINAtShp.mod"
torch.save(model, fine_tuned_model_path)
print(f"Fine-tuned model saved to {fine_tuned_model_path}")

Fine-tuned model saved to /home/j597s263/scratch/j597s263/Models/Resnet/Attack/ResMINAtShp.mod
